# Module scores  

In [ ]:
library_load <- suppressMessages(
    
    suppressWarnings(
    
        list(

            # Seurat 
            library(Seurat), 
            
            # Module score 
            library(UCell), 

            # SingleR
            library(SingleR), 
            library(SingleCellExperiment), 
            
            # Reference
            library(celldex), 

            # Data 
            library(tidyverse), 

            # Pyhton compatibility
            library(reticulate)

        )
    )
)

In [ ]:
# Configure reticulate 
use_condaenv(condaenv="p.3.9.19-FD20200109SPLENO", conda="/nobackup/peer/fdeckert/miniconda3/bin/conda", required=NULL)
py_config()

In [ ]:
options(warn=-1)

In [ ]:
random_seed <- 42
set.seed(random_seed)

In [ ]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [ ]:
# Source files
source("plotting_global.R")

# Parameter settings

In [ ]:
# Files 
raw_rds_file <- "data/BSA_0355_SM01_10x_SPLENO/ANALYSIS/raw.rds"

# Plotting Theme
ggplot2::theme_set(theme_global_set(size_select=1)) # From project global source()

# Import Seurat object

In [ ]:
so <- readRDS(raw_rds_file)

In [ ]:
so <- subset(so, subset=nCount_RNA>=1000)
so <- subset(so, subset=pMt_RNA<=15)

In [ ]:
cnt <- GetAssayData(so, assay="RNA", slot="counts")
cnt <- cnt[rowSums(cnt>=1)>=3, ]
so <- CreateSeuratObject(counts=cnt, meta.data=so@meta.data, project="FD20200109SPLENO")

In [ ]:
so <- NormalizeData(so, normalization.method="LogNormalize", verbose=FALSE)

# Histone gene modules

In [ ]:
features=list(
                               
    msHist_RNA=c("Hist1h1e", "Hist1h1b", "Hist1h2ap", "Hist1h4d", "Hist1h2ae"), 
    msProg_s_RNA=c("Cd34", "Kit", "Gata2"), 
    msProg_t_ery_RNA=c("Hbb-bt"), 
    msProg_t_meg_RNA=c("Pf4")

)

# Compute Module score with UCell 

In [ ]:
# Cell cycle scoring anc class
so <- AddModuleScore_UCell(so, features=features, assay="RNA", slot="counts", name="")

# MinMax scale per sample

In [ ]:
minmax <- function(x) {
    
    x <- (x-min(x))/(max(x)-min(x))
    
    return(x)

}

In [ ]:
# Minmax scale module score per sample
so$msHist_scale_RNA <- dplyr::group_by(so@meta.data, sample_name) %>% dplyr::mutate(msHist_scale_RNA=minmax(msHist_RNA)) %>% dplyr::pull(msHist_scale_RNA)

In [ ]:
options(repr.plot.width=20, repr.plot.height=5)

ggplot(so@meta.data, aes(x=sample_name, y=msHist_RNA)) + geom_jitter() + theme(axis.text.x = element_text(angle=90, vjust=0.5, hjust=1)) 

In [ ]:
options(repr.plot.width=20, repr.plot.height=5)

ggplot(so@meta.data, aes(x=sample_name, y=msHist_scale_RNA)) + geom_jitter() + theme(axis.text.x = element_text(angle=90, vjust=0.5, hjust=1))

# Save results 

In [ ]:
write.csv(so@meta.data[, c("msHist_RNA", "msHist_scale_RNA"), drop=FALSE], "result/module/module.csv", row.names=TRUE)

# Session info

In [ ]:
sessionInfo()